# Difference of estimated noise in #happy and #sad images

The measurement of noise follows the paper Fast Noise Variance Estimation by JohnImmerkær: https://www.sciencedirect.com/science/article/pii/S1077314296900600

It is a fast and simple method for estimating the variance of additive zero mean Gaussian noise in an image. The method can also be used to give a local estimate of the noise variance in the situation in which the noise variance varies across the image. It requires only the use of a 3 􏱗 3 mask followed by a summation over the image or a local neighborhood. A total of 14 integer operations per pixel is necessary. The method performs well for a large range of noise variance values. In textured images or regions, though, the noise estimator perceives thin lines as noise.

The proposed method uses a zero mean operator, which is almost insensitive to image structure. The variance of the output from the operator is an estimate of the noise variance. 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy.lib.stride_tricks import as_strided

# from scipy.signal import convolve2d #
%load_ext memory_profiler
%load_ext line_profiler

## Prepare the image files
- The images are obtained using Instagram Scraper, a command-line application written in Python that scrapes and downloads an instagram user's photos and videos https://github.com/rarcega/instagram-scraper. 
```c
pip install instagram-scraper
instagram-scraper happy --tag -t image -m 1000 -u <username> -p <login password>
```

- Read each .jpg file into a test file 

In [2]:
with open("sad_images.txt", "w") as a:
    for path, subdirs, files in os.walk(r'/Users/ceolwaerc/Dropbox/DAS_project/sad'):
        for filename in files:
            f = os.path.join(path, filename)
            a.write(str(f) + os.linesep)
            
with open("happy_images.txt", "w") as a:
    for path, subdirs, files in os.walk(r'/Users/ceolwaerc/Dropbox/DAS_project/happy'):
        for filename in files:
            f = os.path.join(path, filename)
            a.write(str(f) + os.linesep)

In [2]:
# Open the file of file names and put it in a list

with open('sad_images.txt', 'r') as f:
    sad_list = f.readlines()
sad_list = [x.strip('\n') for x in sad_list] 

with open('happy_images.txt', 'r') as f:
    happy_list = f.readlines()
happy_list = [x.strip('\n') for x in happy_list] 

## Numpy

In [3]:
def greyscale(i):
    '''
    This function turns an image to greyscale
    '''
    img = mpimg.imread(i)
    r, g, b = img[:,:,0], img[:,:,1], img[:,:,2]
    img_grey = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return img_grey

In [4]:
def estimate_noise_np1(img_list, limit=1):
    '''
    This function estimate the local variance of a list of images
    '''
    result = np.array([])
#     img_list = random.shuffle(img_list)
    for i in img_list[0:limit]:
        img_grey= greyscale(i)
        h, w = img_grey.shape
#         matrix = np.zeros((h+4, w+4))
#         matrix[2:-2, 2:-2] = img_grey
        img_grey = np.c_[np.zeros(h), np.zeros(h), img_grey, np.zeros(h), np.zeros(h)] 
        img_grey = np.vstack((np.zeros(w+4), np.zeros(w+4),img_grey, np.zeros(w+4), np.zeros(w+4)))

        N = np.array([[1, -2, 1],
                          [-2, 4, -2],
                          [1, -2, 1]])
        sigma = 0
        for i in range(1, h+3):
            for j in range(1, w+3):
                cell = np.array([img_grey[i-1:i+2, j-1:j+2]])
                cell = np.squeeze(cell, axis=0)
                sigma += abs(np.einsum("ij,ij",N,cell))
        sigma = sigma * np.sqrt(0.5 * np.pi) / (6 * (w-2) * (h-2))
        result = np.append(result, sigma)

    return result
estimate_noise_np1(sad_list, 1)

array([2.14682709])

In [8]:
%timeit estimate_noise_np1(sad_list,1)
%timeit estimate_noise_np1(sad_list,5)
%timeit estimate_noise_np1(sad_list,10)
%timeit estimate_noise_np1(sad_list,20)

In [9]:
%memit estimate_noise_np1(sad_list,5)

peak memory: 149.70 MiB, increment: 52.09 MiB


#### The time it takes to find sigma seems to be proportional to the number of inputs when n is small. However, there are three for loops so when n becomes larger, n^3 will become dominant and greatly increase the time taken. For just 20 images, it already takes 2 minutes to run so we need to seek for an alternative way of doing convolution.

In [23]:
%lprun -f estimate_noise_np1 estimate_noise_np1(sad_list,1)

Timer unit: 1e-06 s

Total time: 5.10089 s
File: <ipython-input-18-b1c2f3b03fa0>
Function: estimate_noise_np1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def estimate_noise_np1(img_list, limit=1):
     2                                               '''
     3                                               This function estimate the local variance of a list of images
     4                                               '''
     5         1          3.0      3.0      0.0      result = []
     6                                           #     img_list = random.shuffle(img_list)
     7         2          3.0      1.5      0.0      for i in img_list[0:limit]:
     8         1      26229.0  26229.0      0.5          img=mpimg.imread(i)
     9         1       8554.0   8554.0      0.2          img_grey = greyscale(img)
    10         1          7.0      7.0      0.0          h, w = img_grey.shape
    11             

## Numpy2

#### In function estimate_noise_np1, forming the submatrix of 3 by 3 and doing mask operation are taking up 94.6% of time and nested for loops would increase time tremendously. A windowed view using stride_tricks enables the function to access the data of the original array in a different way. 
Strides are the number of bytes you need to step in each dimension when traversing the array.

In [5]:
# https://stackoverflow.com/questions/19414673/in-numpy-how-to-efficiently-list-all-fixed-size-submatrices
# -  strides explantion: https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.ndarray.strides.html
from numpy.lib.stride_tricks import as_strided
def estimate_noise_np2(img_list, limit):
    result = np.array([])
    for i in img_list[0:limit]:
        img=mpimg.imread(i)
        img_grey = 0.2989 * img[:,:,0] + 0.5870 * img[:,:,1] + 0.1140 * img[:,:,2]
        h, w = img_grey.shape
        img_m = np.zeros((h+4, w+4))
        img_m[2:-2, 2:-2] = img_grey
        N = np.array([[1, -2, 1],
                      [-2, 4, -2],
                      [1, -2, 1]]).astype(np.int16)
#     a = np.c_[np.zeros(h), np.zeros(h), a, np.zeros(h), np.zeros(h)] 
#     img_grey = np.vstack((np.zeros(w+4), np.zeros(w+4),a, np.zeros(w+4), np.zeros(w+4)))
        s = (3,3) + tuple(np.subtract((h+4, w+4), (3,3)) + 1)
        strd = np.lib.stride_tricks.as_strided
        subM = strd(img_m, shape = s, strides = img_m.strides * 2)
        sigma = np.sum(np.absolute(np.einsum('ij,ijkl->kl', N, subM))) * np.sqrt(0.5 * np.pi) / (6 * (w-2) * (h-2))
        result = np.append(result, sigma)
    return result

In [50]:
estimate_noise_np2(sad_list, 1)

array([2.14682709])

In [13]:
%timeit estimate_noise_np2(sad_list, 1)
%timeit estimate_noise_np2(sad_list, 10)
%timeit estimate_noise_np2(sad_list, 100)

35.3 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
798 ms ± 9.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
8.37 s ± 317 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Time taken still seems to be proportional to the number of images, but it takes much less time to process 10 images, 884ms compared to 60s. 

In [9]:
%memit estimate_noise_np2(sad_list,5)

peak memory: 185.61 MiB, increment: 79.86 MiB


In [12]:
%lprun -f estimate_noise_np2 estimate_noise_np2(sad_list, 1)

Timer unit: 1e-06 s

Total time: 0.061608 s
File: <ipython-input-10-7ef0d47b3356>
Function: estimate_noise_np2 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def estimate_noise_np2(img_list, limit):
     5         1          7.0      7.0      0.0      result = []
     6         2          8.0      4.0      0.0      for i in img_list[0:limit]:
     7         1      30722.0  30722.0     49.9          img=mpimg.imread(i)
     8         1       3841.0   3841.0      6.2          img_grey = greyscale(img)
     9         1          4.0      4.0      0.0          h, w = img_grey.shape
    10         1        285.0    285.0      0.5          img_m = np.zeros((h+4, w+4))
    11         1       1477.0   1477.0      2.4          img_m[2:-2, 2:-2] = img_grey
    12         1         10.0     10.0      0.0          N = np.array([[1, -2, 1],
    13         1          1.0      1.0      0.0                        [-2, 4, -2],
 

## Numba

In [6]:
from numba import jit

@jit
def estimate_noise_nb(img_list, limit):
    result = np.array([])
    for i in img_list[0:limit]:
        img=mpimg.imread(i)
        img_grey = 0.2989 * img[:,:,0] + 0.5870 * img[:,:,1] + 0.1140 * img[:,:,2]
        h, w = img_grey.shape
        img_m = np.zeros((h+4, w+4))
        img_m[2:-2, 2:-2] = img_grey
        N = np.array([[1, -2, 1],
                      [-2, 4, -2],
                      [1, -2, 1]]).astype(np.int16)

        s = (3,3) + tuple(np.subtract((h+4, w+4), (3,3)) + 1)
        strd = np.lib.stride_tricks.as_strided
        subM = strd(img_m, shape = s, strides = img_m.strides * 2)
        sigma = np.sum(np.absolute(np.einsum('ij,ijkl->kl', N, subM))) * np.sqrt(0.5 * np.pi) / (6 * (w-2) * (h-2))
        result = np.append(result, sigma)
    return result

In [41]:
%timeit estimate_noise_nb(sad_list,1)

38.6 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Cython

In [7]:
%load_ext cython

In [7]:
img = mpimg.imread(sad_list[0])
img.dtype

dtype('uint8')

In [8]:
%%cython 

cimport cython
cimport numpy as c_np
# from libc.math cimport round,sqrt
# from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
def estimate_noise(float[:,:,:] img):
    import numpy as np
    cdef int h=img.shape[0]
    cdef int w=img.shape[1]
    cdef int a,b, k, l
    cdef double[:,:] img_grey = np.empty((h,w), dtype=np.double)
    cdef double[:,:] img_m = np.empty((h+4,w+4), dtype=np.double)
#     cdef double[:] result = np.array([]) 
    cdef int[3][3] N= [[1, -2, 1],[-2, 4, -2],[1, -2, 1]]
    cdef double[:, :, :, :] subM
    cdef float s, sigma
    
    for a in range(h):
        for b in range(w):
            img_grey[a,b]=0.299*img[a,b,0]+0.587*img[a,b,1]+0.114*img[a,b,2]
    img_m[2:-2, 2:-2] = img_grey
    for i in range(1,h+3):
        for j in range(1, w+3):
            s = img_m[i-1,j-1]*1 + img_m[i-1,j]*-2 + img_m[i-1,j+1]*1 + img_m[i,j-1]*-2 + img_m[i,j]*4 + img_m[i,j+1]*-2 + img_m[i+1,j-1]*1 + img_m[i+1,j]*-2 + img_m[i+1,j+1]*1
            sigma += abs(s)
    return sigma
#     return grey

CompileError: command 'gcc' failed with exit status 1

In [5]:
# def estimate_noise0(img):
#     '''
#     This function estimate the local variance of image 
#     '''
#     img_grey = greyscale(img)
#     h, w = img_grey.shape
#     N = [[1, -2, 1],
#         [-2, 4, -2],
#         [1, -2, 1]]
#     sigma = np.sum(np.absolute(convolve2d(img_grey, N)))
#     sigma = sigma * np.sqrt(0.5 * np.pi) / (6 * (w-2) * (h-2))
    
#     return sigma